In [12]:
import time
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd




In [89]:
import pickle
filename = 'reviews_newest_346'
outfile = open(filename,'wb')
pickle.dump(review_dict,outfile)
outfile.close()

In [14]:
pickle_in = open("reviews_100","rb")
review_dict = pickle.load(pickle_in)

In [15]:
test_df = pd.DataFrame(review_dict)

In [16]:
test_df.tail()

,comapny_source,company_loc,rev_comp_rating,rev_comp_reviews,rev_comp_url,rev_company_name,userUrl
5273,Exceed Physical Culture,"97 Reade St New York, NY 10013",5.0,Love this place! Always a great workout. Pros:...,https://www.yelp.com/biz/exceed-physical-cultu...,Exceed Physical Culture,https://www.yelp.com/user_details?userid=tB1nj...
5274,Exceed Physical Culture,"2130 Broadway Ave New York, NY 10023",3.0,Today was my first time going to the UWS locat...,https://www.yelp.com/biz/flywheel-sports-upper...,Flywheel Sports Upper West Side,https://www.yelp.com/user_details?userid=tB1nj...
5275,Exceed Physical Culture,"182 Mulberry St New York, NY 10012",5.0,Went to the Speed Lab class a few days ago for...,https://www.yelp.com/biz/soho-strength-lab-new...,SoHo Strength Lab,https://www.yelp.com/user_details?userid=tB1nj...
5276,Exceed Physical Culture,"1 York St New York, NY 10013",4.0,Great studio! Instructors are ass kickers (in...,https://www.yelp.com/biz/barrys-bootcamp-tribe...,Barry’s Bootcamp Tribeca,https://www.yelp.com/user_details?userid=tB1nj...
5277,Exceed Physical Culture,"150 Amsterdam Ave New York, NY 10023",5.0,"Obsessed with this place!! Clean, spacious, a...",https://www.yelp.com/biz/flywheel-sports-linco...,Flywheel Sports - Lincoln Square,https://www.yelp.com/user_details?userid=tB1nj...


In [2]:
def comp_info_and_users(url):
    user_links = []
    page_options = []
    time.sleep(1)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    
    #find title--multiple h1 given for pages, so join them 
    title = soup.find('div',{'class':'biz-page-header'}).findAll('h1')
    company_source_name = ' '.join([x.text.strip() for x in title]).strip()

    #find all the links if multiple pages are given
    match = soup.findAll('div', {'class':'page-option'})# find all pages listed
    page_options.extend([x.a['href'] for x in match[1:]])
    time.sleep(1)
    
    #get users who've left comments on the first page
    user_match = soup.findAll('ul',{'class':'user-passport-info'})
    user_links.extend(['https://www.yelp.com'+x.a['href'] for x in user_match])
    
    #loop through review pages and get all users
    if len(page_options) >= 1:
        for url in page_options:
            time.sleep(1)
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'lxml')

            user_match = soup.findAll('ul',{'class':'user-passport-info'})
            user_links.extend(['https://www.yelp.com'+x.a['href'] for x in user_match])
        
    return company_source_name,user_links
    
    
    

In [3]:
def request_page(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    return soup

def get_reviews(i,soup,r_dict,company_source_name):
    review_headers = soup.findAll('a',{'class':'biz-name js-analytics-click'})
    r_dict['rev_comp_url'].extend(['https://www.yelp.com'+x['href'] for x in review_headers])
    r_dict['rev_company_name'].extend([x.text for x in review_headers])

    reviews_divs = soup.findAll('div',{'review'})
    one,two,three = ('<address>\n','\n    </address>','<br/>')
    r_dict['company_loc'].extend([' '.join(str(x.find('address')).replace(one,'')
                              .replace(two,'').strip().split(three)) for x in reviews_divs])
    r_dict['rev_comp_rating'].extend([float(x.find('div',{'class':'rating-large'})['title'][:3]) 
                                       for x in reviews_divs])
    
    reviews_4_comp = [x.p.text for x in reviews_divs]
    r_dict['rev_comp_reviews'].extend(reviews_4_comp)
    r_dict['userUrl'].extend([i]*len(reviews_4_comp))
    r_dict['comapny_source'].extend([company_source_name]*len(reviews_4_comp))
    
    
    

In [4]:
def check_user_link(user_link,r_dict):
        time.sleep(1)
        driver = webdriver.Chrome('/Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver') 
        time.sleep(1)
        driver.get(user_link)
        time.sleep(1)
        city = driver.find_element_by_class_name('user-location').text
        if city in ('Queens, NY','Manhattan, NY','New York, NY','Brooklyn, NY','Manhattan, New York, NY'):
            time.sleep(1)
            driver.find_element_by_link_text('Reviews').click()
            time.sleep(1)
            driver.find_element_by_link_text('All Categories').click()
            time.sleep(1)
            driver.find_element_by_partial_link_text('Fitness & Instruction').click()
            time.sleep(1)
            reviews = driver.find_elements_by_class_name('review')
            if len(reviews)>1:
                print(f'just did reviewer {counter} for company {comp_num_curr}')
                counter += 1
                time.sleep(1)
                return driver.current_url
    

In [60]:
def reviews_top_pages_test(url,r_dict,review_num,driver):
    company_source_name,user_links = comp_info_and_users(url)
    
    if len(user_links) < 2:
        return 

    counter = 1
    
    for i in user_links:
        time.sleep(1)
        if counter > review_num:
            break
        if i not in r_dict['userUrl']:
            time.sleep(1)
            driver.get(i)
            time.sleep(1)
            try:
                city = driver.find_element_by_class_name('user-location').text
                if city in ('Queens, NY','Manhattan, NY','New York, NY','Brooklyn, NY','Manhattan, New York, NY'):
                    time.sleep(1)
                    driver.find_element_by_link_text('Reviews').click()
                    time.sleep(1)
                    driver.find_element_by_link_text('All Categories').click()
                    time.sleep(1)
                    driver.find_element_by_partial_link_text('Fitness & Instruction').click()
                    time.sleep(1)
                    reviews = driver.find_elements_by_class_name('review')
                    if len(reviews)>1:
                        comp_num_curr = len(set(r_dict['comapny_source']))
                        print(f'just did reviewer {counter} for company {comp_num_curr}')
                        counter += 1
                        time.sleep(1)
                        currentURL = driver.current_url
                        soup = request_page(currentURL)
                        #userName = soup.find('div',{'class':'user-profile_info arrange_unit'}).h1.text
                        page_options = soup.findAll('div',{'class':'page-option'})
                        pages = [x.a['href'] for x in page_options[1:]]
                        time.sleep(1)
                        get_reviews(i,soup,r_dict,company_source_name)

                        for url in pages:
                            soup = request_page(url)
                            get_reviews(i,soup,r_dict,company_source_name)
                    else:
                        time.sleep(1)

                else:
                    time.sleep(1)
            except Exception as e:
                print(e)
                time.sleep(1)
                continue
        else:
            time.sleep(1)
            continue
            
    comp_num_curr = len(set(r_dict['comapny_source']))         
    return f'done with {comp_num_curr}'

In [6]:
#company and not 'comapny_source' 

def get_all_reviews(url_1,r_dict,review_num,company_count,starter_index):
    
    driver = webdriver.Chrome('/Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver') 
    reviews_top_pages_test(url_1,r_dict,review_num,driver)
    
    ok_locs = ['Brooklyn,NY', 'York,NY', 'Queens,NY','Bushwick,NY','Manhattan,NY','Astoria,NY ']
    
    for i in zip(r_dict['rev_company_name'][starter_index:],r_dict['rev_comp_url'][starter_index:],r_dict['company_loc'][starter_index:]):
        comp_num_curr = len(set(r_dict['comapny_source']))
        if comp_num_curr > company_count:
            break
        if (i[0] not in r_dict['comapny_source']) and ('CLOSED' not in i[0]) \
        and ((''.join(i[2].split(' ')[-3:-1])) in ok_locs) and ('MOVED' not in i[0]):
            if comp_num_curr % 100 == 0:
                filename = f'reviews_{comp_num_curr}'
                outfile = open(filename,'wb')
                pickle.dump(review_dict,outfile)
                outfile.close()
                time.sleep(1)
                for col in list(r_dict.keys()):
                    print(len(r_dict[col]))
                reviews_top_pages_test(i[1],r_dict,review_num,driver)
            else:
                time.sleep(1)
                reviews_top_pages_test(i[1],r_dict,review_num,driver)
                        
    driver.close()
    return pd.DataFrame(r_dict)
    

In [ ]:
# url_test = 'https://www.yelp.com/biz/peloton-new-york?osq=peloton'
# url_2 = 'https://www.yelp.com/biz/crunch-new-york-new-york'
# url_3 = 'https://www.yelp.com/biz/breathe-salt-rooms-new-york-2'
# url_4 = 'https://www.yelp.com/biz/jazzercise-lower-manhattan-new-york-2'
# url_5 = 'https://www.yelp.com/biz/moonheart-healing-arts-center-new-york'
# url_6 = 'https://www.yelp.com/biz/chaisefitness-new-york'
# url_7 = 'https://www.yelp.com/biz/tracy-anderson-method-new-york-2'
# url_8 = 'https://www.yelp.com/biz/vanderbilt-tennis-club-at-grand-central-terminal-new-york'
url_9 = 'https://www.yelp.com/biz/epic-hybrid-training-new-york-5'

reviews_df = get_all_reviews(url_9,review_dict,10,999,1344)

In [9]:
for i in list(review_dict.keys()):
    print(len(review_dict[i]))

NameError: name 'review_dict' is not defined

In [93]:
lets_go = {'comapny_source':['Peloton','Peloton','Peloton','Peloton','Peloton'],'company_loc':['370 Canal St New York, NY 10013',
                                                                                        '90 E 10th St New York, NY 10003',
                                                                                        '1841 Broadway New York, NY 11023',
                                                                                        '19 W 45th St New York, NY 10036',
                                                                                        '140 W 23rd St New York, NY 10011'],
    'rev_comp_rating':[3.0, 2.0, 3.0, 4.0, 4.0],'rev_comp_reviews':['rev','rev','rev','rev','rev'],
    'rev_comp_url':['https://www.yelp.com/biz/planet-fitness-manhattan-canal-st-ny-new-york',
                    'https://www.yelp.com/biz/montauk-salt-cave-new-york',
                    'https://www.yelp.com/biz/pure-barre-new-york-columbus-circle-60th-and-broadway-new-york',
                    'https://www.yelp.com/biz/return-to-life-center-pilates-and-functional-movement-new-york-2',
                    'https://www.yelp.com/biz/peloton-new-york'],'rev_company_name':['Planet Fitness - Manhattan - Canal St - NY',
                                                                                     'Montauk Salt Cave',
                                                                                     'Pure Barre - New York Columbus Circle - 60th & Broadway',
                                                                                     'Return To Life Center - Pilates and Functional Movement',
                                                                                     'Peloton'],'userUrl':['https://www.yelp.com/user_details?userid=exPhuONOpfCptZ35Ur3bcQ',
                                                                                                           'https://www.yelp.com/user_details?userid=exPhuONOpfCptZ35Ur3bcQ',
                                                                                                           'https://www.yelp.com/user_details?userid=exPhuONOpfCptZ35Ur3bcQ',
                                                                                                           'https://www.yelp.com/user_details?userid=exPhuONOpfCptZ35Ur3bcQ',
                                                                                                           'https://www.yelp.com/user_details?userid=exPhuONOpfCptZ35Ur3bcQ']}


In [94]:
lets_go

{'comapny_source': ['Peloton', 'Peloton', 'Peloton', 'Peloton', 'Peloton'],
 'company_loc': ['370 Canal St New York, NY 10013',
  '90 E 10th St New York, NY 10003',
  '1841 Broadway New York, NY 11023',
  '19 W 45th St New York, NY 10036',
  '140 W 23rd St New York, NY 10011'],
 'rev_comp_rating': [3.0, 2.0, 3.0, 4.0, 4.0],
 'rev_comp_reviews': ['rev', 'rev', 'rev', 'rev', 'rev'],
 'rev_comp_url': ['https://www.yelp.com/biz/planet-fitness-manhattan-canal-st-ny-new-york',
  'https://www.yelp.com/biz/montauk-salt-cave-new-york',
  'https://www.yelp.com/biz/pure-barre-new-york-columbus-circle-60th-and-broadway-new-york',
  'https://www.yelp.com/biz/return-to-life-center-pilates-and-functional-movement-new-york-2',
  'https://www.yelp.com/biz/peloton-new-york'],
 'rev_company_name': ['Planet Fitness - Manhattan - Canal St - NY',
  'Montauk Salt Cave',
  'Pure Barre - New York Columbus Circle - 60th & Broadway',
  'Return To Life Center - Pilates and Functional Movement',
  'Peloton'],
 'us

In [90]:
test = pd.DataFrame(review_dict)

In [91]:
test.tail(10)

,comapny_source,company_loc,rev_comp_rating,rev_comp_reviews,rev_comp_url,rev_company_name,userUrl
12533,Refine Method,"23 W 73rd St New York, NY 10023",2.0,"I went to this gym awhile ago with a friend, p...",https://www.yelp.com/biz/new-york-sports-clubs...,New York Sports Clubs,https://www.yelp.com/user_details?userid=SnHN0...
12534,Refine Method,"117 W 72nd St Manhattan, NY 10023",5.0,Excellent fitness studio. An excellent high in...,https://www.yelp.com/biz/refine-method-manhatt...,Refine Method,https://www.yelp.com/user_details?userid=zXIBH...
12535,Refine Method,"2210 S 1st St Austin, TX 78704",5.0,"Cody Butler, personal trainer and founder, is ...",https://www.yelp.com/biz/heat-bootcamp-south-a...,HEAT Bootcamp South Austin,https://www.yelp.com/user_details?userid=zXIBH...
12536,Refine Method,"117 W 72nd St Manhattan, NY 10023",5.0,I had been looking for a good workout on the u...,https://www.yelp.com/biz/refine-method-manhatt...,Refine Method,https://www.yelp.com/user_details?userid=2JlmJ...
12537,Refine Method,"340 E 71st St New York, NY 10021",5.0,I describe this to friends as sort of a more c...,https://www.yelp.com/biz/refine-method-new-york-4,Refine Method,https://www.yelp.com/user_details?userid=2JlmJ...
12538,Refine Method,"204 W 77th St New York, NY 10024",2.0,I went to Pure for about 18 months. The classe...,https://www.yelp.com/biz/pure-yoga-new-york-3,Pure Yoga,https://www.yelp.com/user_details?userid=2JlmJ...
12539,WundaBar Pilates,"21 Thompson St New York, NY 10013",1.0,Instructor never showed up. I love the workout...,https://www.yelp.com/biz/wundabar-pilates-new-...,WundaBar Pilates,https://www.yelp.com/user_details?userid=BlyJB...
12540,WundaBar Pilates,"636 Broadway New York, NY 10012",5.0,When I first tried Work Train Fight I took Box...,https://www.yelp.com/biz/work-train-fight-new-...,Work Train Fight,https://www.yelp.com/user_details?userid=BlyJB...
12541,WundaBar Pilates,"21 Thompson St New York, NY 10013",5.0,Wundabar is the best!! I loved them in LA and...,https://www.yelp.com/biz/wundabar-pilates-new-...,WundaBar Pilates,https://www.yelp.com/user_details?userid=oazgu...
12542,WundaBar Pilates,"12416 Ventura Blvd Studio City, CA 91604",5.0,I absolutely love Wundabar... and this is comi...,https://www.yelp.com/biz/wundabar-pilates-stud...,WundaBar Pilates,https://www.yelp.com/user_details?userid=oazgu...


In [93]:
test_df.iloc[1341:,:]

,comapny_source,company_loc,rev_comp_rating,rev_comp_reviews,rev_comp_url,rev_company_name,userUrl
1341,Mile High Run Club,"21 Thompson St New York, NY 10013",5.0,"While I'm not stranger to working out, I'm fai...",https://www.yelp.com/biz/wundabar-pilates-new-...,WundaBar Pilates,https://www.yelp.com/user_details?userid=0M_kM...
1342,Mile High Run Club,"38 W 38th St New York, NY 10018",5.0,Epic is consistent in their excellence! I've b...,https://www.yelp.com/biz/epic-hybrid-training-...,EPIC Hybrid Training - CLOSED,https://www.yelp.com/user_details?userid=0M_kM...
1343,Mile High Run Club,"226 E 54th New York, NY 10022",5.0,"Everyone else pretty much summed it up, but th...",https://www.yelp.com/biz/epic-hybrid-training-...,Epic Hybrid Training,https://www.yelp.com/user_details?userid=0M_kM...
1344,Mile High Run Club,"222 E 34th St New York, NY 10016",1.0,I stuck it out for a while. I'm sad to leave m...,https://www.yelp.com/biz/crunch-fitness-34th-s...,Crunch Fitness - 34th Street,https://www.yelp.com/user_details?userid=0M_kM...
1345,Mile High Run Club,"222 E 34th St New York, NY 10016",1.0,I already ranted on the Crunch page. I was a v...,https://www.yelp.com/biz/club-h-fitness-new-yo...,Club H Fitness - CLOSED,https://www.yelp.com/user_details?userid=0M_kM...
1346,Mile High Run Club,"24 W 25th St New York, NY 10010",5.0,"I'm really not a runner, but trying to get mor...",https://www.yelp.com/biz/mile-high-run-club-ne...,Mile High Run Club,https://www.yelp.com/user_details?userid=GZs1x...
1347,Mile High Run Club,"39 W 21st St New York, NY 10010",5.0,I came late and they were super accommodating ...,https://www.yelp.com/biz/flywheel-sports-flati...,Flywheel Sports - Flatiron,https://www.yelp.com/user_details?userid=GZs1x...
1348,Mile High Run Club,"28 E 4th St New York, NY 10003",5.0,"I have been on ClassPass since January, it's n...",https://www.yelp.com/biz/mile-high-run-club-ne...,Mile High Run Club,https://www.yelp.com/user_details?userid=GZs1x...
1349,Mile High Run Club,"1429 2nd Ave New York, NY 10021",5.0,The bathrooms have been remodeled and now it's...,https://www.yelp.com/biz/equinox-east-74th-str...,Equinox East 74th Street,https://www.yelp.com/user_details?userid=GZs1x...
1350,Mile High Run Club,"419 Lafayette St New York, NY 10003",1.0,"I've really been enjoying Barry's, but this gu...",https://www.yelp.com/biz/barrys-bootcamp-noho-...,Barry’s Bootcamp NoHo,https://www.yelp.com/user_details?userid=GZs1x...


In [95]:
test.iloc[1343,4]

'https://www.yelp.com/biz/epic-hybrid-training-new-york-5'

In [162]:
page = requests.get('https://www.yelp.com/biz/tracy-anderson-new-york-2')
soup = BeautifulSoup(page.content, 'lxml')
title = soup.find('div',{'class':'biz-page-header'}).findAll('h1')
company_source_name = ' '.join([x.text.strip() for x in title]).strip()
company_source_name

'Tracy Anderson'

In [161]:
company_source_name

'Moonheart Healing Arts Center'

In [94]:
test_df[test_df['comapny_source']=='Epic Hybrid Training']

,comapny_source,company_loc,rev_comp_rating,rev_comp_reviews,rev_comp_url,rev_company_name,userUrl


In [92]:
test_df[test_df['rev_company_name']=='WundaBar Pilates']

,comapny_source,company_loc,rev_comp_rating,rev_comp_reviews,rev_comp_url,rev_company_name,userUrl
1341,Mile High Run Club,"21 Thompson St New York, NY 10013",5.0,"While I'm not stranger to working out, I'm fai...",https://www.yelp.com/biz/wundabar-pilates-new-...,WundaBar Pilates,https://www.yelp.com/user_details?userid=0M_kM...
1418,Pop Physique,"21 Thompson St New York, NY 10013",4.0,"Small, one-room studio with ten Wundaformer Pi...",https://www.yelp.com/biz/wundabar-pilates-new-...,WundaBar Pilates,https://www.yelp.com/user_details?userid=j0eBR...
2379,Uplift Studios,"21 Thompson St New York, NY 10013",3.0,"***Hands-on instructor, Challenging, Limited f...",https://www.yelp.com/biz/wundabar-pilates-new-...,WundaBar Pilates,https://www.yelp.com/user_details?userid=Cpwjs...
2639,TITLE Boxing Club,"21 Thompson St New York, NY 10013",5.0,WundaBar was the first official pilates class ...,https://www.yelp.com/biz/wundabar-pilates-new-...,WundaBar Pilates,https://www.yelp.com/user_details?userid=iQOH7...
4045,New York Pilates - Soho,"21 Thompson St New York, NY 10013",5.0,"I'm not really a workout girl, but this winter...",https://www.yelp.com/biz/wundabar-pilates-new-...,WundaBar Pilates,https://www.yelp.com/user_details?userid=3DeCq...
4059,New York Pilates - Soho,"21 Thompson St New York, NY 10013",5.0,"This is pilates, but taken to a whole other le...",https://www.yelp.com/biz/wundabar-pilates-new-...,WundaBar Pilates,https://www.yelp.com/user_details?userid=kVEG2...
4081,New York Pilates - Soho,"21 Thompson St New York, NY 10013",4.0,Definitely a fan! I did my first pilates class...,https://www.yelp.com/biz/wundabar-pilates-new-...,WundaBar Pilates,https://www.yelp.com/user_details?userid=xlVer...
4102,New York Pilates,"21 Thompson St New York, NY 10013",5.0,I was sooo excited when I discovered this plac...,https://www.yelp.com/biz/wundabar-pilates-new-...,WundaBar Pilates,https://www.yelp.com/user_details?userid=DBZ1N...
4624,Surfset New York City,"21 Thompson St New York, NY 10013",4.0,This is a great workout for my active recovery...,https://www.yelp.com/biz/wundabar-pilates-new-...,WundaBar Pilates,https://www.yelp.com/user_details?userid=Ss1uC...
4875,Body By Simone,"21 Thompson St New York, NY 10013",2.0,I first tried Wunda when it was just opening u...,https://www.yelp.com/biz/wundabar-pilates-new-...,WundaBar Pilates,https://www.yelp.com/user_details?userid=hl5Eq...


In [55]:
test.rev_comp_url.iloc[457]

'https://www.yelp.com/biz/tracy-anderson-new-york-2'

In [123]:
list(test.comapny_source)

['Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Peloton',
 'Pe

In [124]:
for i in list(test.comapny_source):
    print(i)

Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Peloton
Planet Fitness - Manhattan - Canal St - NY
Planet Fitness - Manhattan - Canal St - NY
Planet Fitness - Manhattan - Canal St - NY
Planet Fitness - Manhattan - Canal St - NY
Planet Fitness - Manhattan - Canal St - NY
P

YogaWorks Soho
YogaWorks Soho
YogaWorks Soho
YogaWorks Soho
YogaWorks Soho
YogaWorks Soho
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre - New York Tribeca
Pure Barre

Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
Equinox Rockefeller Center
E

Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Uplift Studios
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Barre3 - West Village
Ba

In [170]:
review_dict['comapny_source'][450:]

['Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 'Exhale Upper East Side',
 

In [29]:
len(review_dict['comapny_source'])

7636